In [1]:
import random
import math


In [2]:
import os
import subprocess
from subprocess import Popen, PIPE, STDOUT
from multiprocessing import Process
import time
from timeit import default_timer as timer
import multiprocessing as mp

# output_folder = "output_plane"
# job_filename = "job_plane.txt"
# progress_filename = "progress_plane.txt"
# folder_id = "02691156" # plane

output_folder = "output_car"
progress_filename = "progress_car.txt"
job_filename = "job_car.txt"
folder_id = "02958343"

# output_folder = "output_chair"
# progress_filename = "progress_chair.txt"
# job_filename = "job_chair.txt"
# folder_id = "03001627" # chair


# shapnet_path = "/NAS/data/shapenet/ShapeNetCore.v2"
# render_path = "/NAS/home/6dof/models/research/keypointnet/tools/"

shapnet_path = "/home/paperspace/zen/6dof/6dof_data/ShapeNetCore.v1"
render_path = "/home/paperspace/zen/6dof/models/research/keypointnet/tools/"


batch_size = 8
f = open( render_path + job_filename,"r") 
f_save = open( render_path + progress_filename,"r+") 
ids = []
done_ids = []
for s in f:
    ids.append(s.split(",")[1])
ids = ids[1:]
for s in f_save:
    done_ids.append(s.strip())
f_save.close()



In [3]:
dirs = os.listdir(shapnet_path)
all_ids = []
for i in dirs:
    file_path = os.path.join(shapnet_path,i)
    if os.path.isdir(file_path):
        all_ids += os.listdir(file_path)


In [34]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import os
from scipy import misc
import tensorflow as tf
import argparse
import sys

# output_dir = "/home/paperspace/zen/6dof/6dof_data/zen_plane/"
# input_dir = "/home/paperspace/zen/6dof/models/research/keypointnet/tools/output_plane/02691156/"

def get_matrix(lines):
    return np.array([[float(y) for y in x.strip().split(" ")] for x in lines])


def read_model_view_matrices(filename):
    with open(filename, "r") as f:
        lines = f.readlines()
    return get_matrix(lines[:4]), get_matrix(lines[4:])


def bytes_feature(values):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))


def generate(files, output_dir, input_dir, chunk_size = 40):
    file_chunks = [files[i:i + chunk_size] for i in range(0, len(files), chunk_size)]
    for i in range(len(file_chunks)):
        files = file_chunks[i]
        record_name = output_dir + '{0:04}'.format(i) + ".tfrecord"
        with tf.python_io.TFRecordWriter(record_name) as tfrecord_writer:
            with tf.Graph().as_default():
                im0 = tf.placeholder(dtype=tf.uint8)
                im1 = tf.placeholder(dtype=tf.uint8)
                encoded0 = tf.image.encode_png(im0)
                encoded1 = tf.image.encode_png(im1)
                with tf.Session() as sess:
                    for file_name in files:
                        count = 0
                        indir = input_dir + file_name  + "/"
                        while tf.gfile.Exists(indir + "%06d.txt" % count):
                            image0 = misc.imread(indir + "%06d.png" % (count * 2))
                            image1 = misc.imread(indir + "%06d.png" % (count * 2 + 1))

                            mat0, mat1 = read_model_view_matrices(
                                indir + "%06d.txt" % count)

                            mati0 = np.linalg.inv(mat0).flatten()
                            mati1 = np.linalg.inv(mat1).flatten()
                            mat0 = mat0.flatten()
                            mat1 = mat1.flatten()

                            st0, st1 = sess.run([encoded0, encoded1],
                                                feed_dict={im0: image0, im1: image1})

                            example = tf.train.Example(features=tf.train.Features(feature={
                                'img0': bytes_feature(st0),
                                'img1': bytes_feature(st1),
                                'mv0': tf.train.Feature(
                                    float_list=tf.train.FloatList(value=mat0)),
                                'mvi0': tf.train.Feature(
                                    float_list=tf.train.FloatList(value=mati0)),
                                'mv1': tf.train.Feature(
                                    float_list=tf.train.FloatList(value=mat1)),
                                'mvi1': tf.train.Feature(
                                    float_list=tf.train.FloatList(value=mati1)),
                            }))

                            tfrecord_writer.write(example.SerializeToString())
                            count += 1
                        print("finished: " + file_name)
    
# if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('-t', '--target', dest='target',
#                         required=True,
#                         help='target set: car, plane, chair')


#     if '--' not in sys.argv:
#         parser.print_help()
#         exit(1)

#     argv = sys.argv[sys.argv.index('--') + 1:]
#     args, _ = parser.parse_known_args(argv)
#     if args.target == 'car':
#         output_dir = "/NAS/home/shapenet_rendering/shapenet_car/cars_with_keypoints/"
#         input_dir = "/NAS/home/shapenet_rendering/shapenet_car/02958343/"
#     elif args.target == "plane":
#         output_dir = "/NAS/home/shapenet_rendering/shapenet_plane/planes_with_keypoints/"
#         input_dir = "/NAS/home/shapenet_rendering/shapenet_plane/02691156/"
#     elif args.target == "chair":
#         output_dir = "/NAS/home/shapenet_rendering/shapenet_chair/chairs_with_keypoints/"
#         input_dir = "/NAS/home/shapenet_rendering/shapenet_chair/03001627/"
#     else:
#         parser.print_help()
#         exit(1)
output_dir = "/home/paperspace/zen/6dof/6dof_data/zen_plane/"
input_dir = "/home/paperspace/zen/6dof/models/research/keypointnet/tools/output_plane/02691156/"
files = os.listdir(input_dir)

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
generate(files, output_dir, input_dir, 40)


    


/home/paperspace/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/paperspace/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


finished: a2b758aa5d51642bd32761b337f8b72a
finished: 2cf6583a74dc4d1e373ed874fe97420b
finished: 14282db0ca4238429a6e43b878d5b335
finished: 24fbe7a49fd786c5fb5c1b0f759e2bc1


KeyboardInterrupt: 

In [20]:
chunk_size = 40
file_chunks = [files[i:i + chunk_size] for i in range(0, len(files), chunk_size)]
for i in range(len(file_chunks)):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [35]:
files = [output_dir + i for i in os.listdir(output_dir)]
for fn in files:
    c = 0
    for record in tf.python_io.tf_record_iterator(fn):
        c += 1
    print(c)

491
